# LASSO (coordinate descent)

In this notebook, you will implement your very own LASSO solver via coordinate descent. You will:
* Write a function to normalize features
* Implement coordinate descent for LASSO
* Explore effects of L1 penalty

## Yes, this again

In [1]:
import sklearn
import pandas

## Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located. *Wow.*

In [2]:
full_data = pandas.read_csv("/content/drive/MyDrive/FUNIX Progress/MLP302x_1.1-A_EN/data/kc_house_data.csv", index_col=0)
# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to int, before using it below
full_data['floors'] = full_data['floors'].astype(int) 

If we want to do any "feature engineering" like creating new features or adjusting existing ones we should do this directly using the DataFrame as seen in the notebook of Lab 2 (*lab-2.ipynb*). For this notebook, however, we will work with the existing features.

## Import useful functions from previous notebook

As in Exercise 1 (*exercise-1.ipynb*), we convert the DataFrame into a 2D Numpy array. Copy and paste `get_numpy_data()` from the exercise there.

In [3]:
import numpy as np

In [4]:
def get_numpy_data(data, features_title, output_title):
    if('constant' not in data):
        data['constant'] = 1 # this is how you add a constant column. Execute only when necessary
    # add the column 'constant' to the front of the features list so that we can extract it along with the others:
    features_title = ['constant'] + features_title # this is how you combine two lists
    # split the data into a sub-DataFrame containing your specified features (including constant).
    features_columns = data[features_title]
    # call it features_columns.

    # the following line will extract the numpy matrix from variable features_columns:
    feature_matrix = features_columns.values
    # retrieve the data associated with the output in a pandas Series
    output_column = data[output_title]
    # call it output_column

    # the following will convert the aforementioned Series into a numpy array
    output_array = output_column.values
    return(feature_matrix, output_array)

Also, copy and paste the `predict_output()` function to compute the predictions for an entire matrix of features given the matrix and the weights:

In [5]:
def predict_output(feature_matrix, weights):
    return(np.dot(feature_matrix, weights))

## Normalize features
In the house dataset, features vary wildly in their relative magnitude: `sqft_living` is very large overall compared to `bedrooms`, for instance. As a result, weight for `sqft_living` would be much smaller than weight for `bedrooms`. This is problematic because "small" weights are dropped first as `l1_penalty` goes up. 

To give equal considerations for all features, we need to **normalize features** as discussed in the lectures: we divide each feature by its 2-norm so that the transformed feature has norm 1.

Let's see how we can do this normalization easily with Numpy: let us first consider a small matrix.

In [6]:
X = np.array([[3.,5.,8.],[4.,12.,15.]])
print(X)

[[ 3.  5.  8.]
 [ 4. 12. 15.]]


Numpy provides a shorthand for computing 2-norms of each column:

In [7]:
norms = np.linalg.norm(X, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print(norms)

[ 5. 13. 17.]


To normalize, apply element-wise division:

In [8]:
print(X / norms) # gives [X[:,0]/norm(X[:,0]), X[:,1]/norm(X[:,1]), X[:,2]/norm(X[:,2])]

[[0.6        0.38461538 0.47058824]
 [0.8        0.92307692 0.88235294]]


Using the shorthand we just covered, write a short function called `normalize_features(feature_matrix)`, which normalizes columns of a given feature matrix. The function should return a pair `(normalized_features, norms)`, where the second item contains the norms of original features. As discussed in the lectures, we will use these norms to normalize the test data in the same way as we normalized the training data. 

In [9]:
def normalize_features(feature_matrix):
    norms = np.linalg.norm(feature_matrix, axis=0)
    normalized_features = feature_matrix / norms
    
    return (normalized_features, norms)

To test the function, run the following:

In [10]:
features, norms = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]))
print(features)
# should print
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print(norms)
# should print
# [5.  10.  15.]

[[0.6 0.6 0.6]
 [0.8 0.8 0.8]]
[ 5. 10. 15.]


## Implementing Coordinate Descent with normalized features

We seek to obtain a sparse set of weights by minimizing the LASSO cost function
<!-- ``` SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|). ``` -->
$cost = \sum (prediction - output)^2 + \lambda * \sum_{i \neq 0} |w_i|$

(By convention, we do not include $w_0$(bias) in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use **coordinate descent**: at each iteration, we will fix all weights but weight `i` and find the value of weight `i` that minimizes the objective. That is, we look for: <br>
<!-- ``` argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ] ``` -->
$argmin_{w_i}(cost_i) = \sum (prediction - output)^2 + \lambda * \sum_{i \neq 0} |w_i| )$

where all weights other than $w_i$(`w[i]`) are held to be constant. We will optimize one $w_i$ at a time, circling through the weights multiple times.  
  1. Pick a coordinate `i`
  2. Compute $w_i$ that minimizes the cost function $cost = (\sum prediction - output) + \lambda * \sum_{i \neq 0} |w_i|$
  3. Repeat Steps 1 and 2 for all coordinates, multiple times

For this notebook, we use **cyclical coordinate descent with normalized features**, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:
<!-- ```
       ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2
``` -->
$w_i = \left\{
\begin{array}{ll}
      \rho_i + \lambda / 2 & \rho_i < -\lambda/2 \\
      0 & -\lambda/2 \leq \rho_i \leq \lambda/2  \\
      \rho_i - \lambda / 2 & \rho_i > \lambda/2 \\
\end{array} 
\right. $

where $\rho_i$(`ro[i]`) is defined as:
<!-- ```ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]. ``` -->
$\rho_i = \sum feature_i * (output - prediction + w_i*feature_i)$

Note that we do not regularize the weight of the constant feature (intercept|bias) $w_0$(`w[0]`), so, for this weight, the update is simply:
<!-- ```w[0] = ro[i]``` -->
$w_0 = \rho_i$

## Effect of L1 penalty

Let us consider a simple model with 2 features:

In [11]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(full_data, simple_features, my_output)

Don't forget to normalize features:

In [12]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)

We assign some random set of initial weights and inspect the values of `ro[i]`:

In [13]:
weights = np.array([1., 4., 1.])

Use `predict_output()` to make predictions on this data.

In [14]:
prediction = predict_output(simple_feature_matrix, weights) 

Compute the values of `ro[i]` for each feature in this simple model, using the formula given above:
<!-- ```ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]. ``` -->
$\rho_i = \sum feature_i * (output - prediction + w_i*feature_i)$

*Hint: You can get a Numpy vector for feature_i using:*
```
simple_feature_matrix[:,i]
```

In [16]:
ro = []
for i in range(len(weights)):
  ro.append(simple_feature_matrix[:,i].dot(output - prediction + simple_feature_matrix[:,i].dot(weights[i])))

In [17]:
ro

[79400300.01452288, 87939470.82325178, 80966698.66623949]

***QUIZ QUESTION***

Recall that, whenever `ro[i]` falls between `-l1_penalty/2` and `l1_penalty/2`, the corresponding weight `w[i]` is sent to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of `l1_penalty` **would not** set `w[1]` zero, but **would** set `w[2]` to zero, if we were to take a step in that coordinate? 

In [19]:
print(2* ro[1])
print(2* ro[2])

175878941.64650357
161933397.33247897


***QUIZ QUESTION***

What range of values of `l1_penalty` would set **both** `w[1]` and `w[2]` to zero, if we were to take a step in that coordinate? 

In [21]:
def in_l1range(value, penalty):
    return ((value >= -penalty/2.) and (value <= penalty/2.))

for l1_penalty in [1.4e8, 1.64e8, 1.73e8, 1.9e8, 2.3e8]:
    print(in_l1range(ro[1], l1_penalty), in_l1range(ro[2], l1_penalty))

False False
False True
False True
True True
True True


So we can say that `ro[i]` quantifies the significance of the i-th feature: the larger `ro[i]` is, the more likely it is for the i-th feature to be retained.

## Single Coordinate Descent Step

Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.

In [ ]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix, weights)
    
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = (feature_matrix[:,i]*(output - prediction + (weights[i]*feature_matrix[:,i]))).sum()

    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
    elif ro_i < -l1_penalty/2.:
        new_weight_i = (ro_i + l1_penalty/2.)
    elif ro_i > l1_penalty/2.:
        new_weight_i = (ro_i - l1_penalty/2.)
    else:
        new_weight_i = 0.
    
    return new_weight_i

To test the function, run the following cell:

In [27]:
# should print 0.425558846691
import math
print(lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]]), 
                                   np.array([1., 1.]), np.array([1., 4.]), 0.1) )

0.4255588466910251


## Cyclical coordinate descent 

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:
1. As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
2. After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to step 1.

Return weights

**IMPORTANT: when computing a new weight for coordinate i, make sure to incorporate the new weights for coordinates 0, 1, ..., i-1. One good way is to update your weights variable in-place. See following pseudocode for illustration.**
```
for i in range(len(weights)):
    old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
    # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
    #     and old values for weight[i], ..., weight[d-1]
    weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
    
    # use old_weights_i to compute change in coordinate
    ...
```

In [29]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    D = feature_matrix.shape[1]
    weights = np.array(initial_weights)
    change = np.array(initial_weights) * 0.0
    converged = False

    while not converged:
        for idx in range(D):
            new_weight = lasso_coordinate_descent_step(idx, feature_matrix,
                                                       output, weights,
                                                       l1_penalty)
            change[idx] = np.abs(new_weight - weights[idx])

            weights[idx] = new_weight
        max_change = max(change)
        
        if max_change < tolerance:
            converged = True
    return weights

Using the following parameters, learn the weights on the sales dataset. 

In [30]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

First create a normalized version of the feature matrix, `normalized_simple_feature_matrix`.

In [31]:
(simple_feature_matrix, output) = get_numpy_data(full_data, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

Then, run your implementation of LASSO coordinate descent:

In [32]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)
prediction = predict_output(normalized_simple_feature_matrix, weights)
rss = np.sum((output - prediction) ** 2)
print("{:.2E}".format(rss))
print(np.array(['constant'] + simple_features)[weights != 0])

1.63E+15
['constant' 'sqft_living']


***QUIZ QUESTIONS***
1. What is the RSS of the learned model on the normalized dataset? (Hint: use the normalized feature matrix when you make predictions.)
2. Which features had weight zero at convergence?

# Evaluating LASSO fit with more features

Let us split the sales dataset into training and test sets.

In [33]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(full_data, train_size=0.8, test_size=0.2, random_state=0)

Let us consider the following set of features.

In [34]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

First, create a normalized feature matrix from the TRAINING data with these features.  (Make you store the norms for the normalization, since we'll use them later)

In [35]:
my_output = 'price'
(feature_matrix, output) = get_numpy_data(train_data, all_features, my_output)
normalized_feature_matrix, norms = normalize_features(feature_matrix)

First, learn the weights with `l1_penalty=1e7`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e7`, you will need them later.

In [ ]:
# Handholding level: nonexistent. Not that you will need any at this point.

***QUIZ QUESTION***

What features had non-zero weight in this case?

In [41]:
initial_weights = np.zeros(len(all_features) + 1)
l1_penalty = 1e7
tolerance = 1.0

weights1e7 = lasso_cyclical_coordinate_descent(normalized_feature_matrix, output,
                                               initial_weights, l1_penalty, tolerance)

feature_list = ['constant'] + all_features
feature_weights1e7 = dict(zip(feature_list, weights1e7))
feature_weights1e7

{'bathrooms': 0.0,
 'bedrooms': 0.0,
 'condition': 0.0,
 'constant': 24625317.015876364,
 'floors': 0.0,
 'grade': 0.0,
 'sqft_above': 0.0,
 'sqft_basement': 0.0,
 'sqft_living': 48292264.535922706,
 'sqft_lot': 0.0,
 'view': 7734349.097161738,
 'waterfront': 3566000.3328484744,
 'yr_built': 0.0,
 'yr_renovated': 0.0}

Next, learn the weights with `l1_penalty=1e8`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e8`, you will need them later.

In [43]:
l1_penalty=1e8
tolerance = 1.0
weights1e8 = lasso_cyclical_coordinate_descent(normalized_feature_matrix, output,
                                               initial_weights, l1_penalty, tolerance)

feature_list = ['constant'] + all_features
feature_weights1e8 = dict(zip(feature_list, weights1e8))
feature_weights1e8

{'bathrooms': 0.0,
 'bedrooms': 0.0,
 'condition': 0.0,
 'constant': 71373534.79051217,
 'floors': 0.0,
 'grade': 0.0,
 'sqft_above': 0.0,
 'sqft_basement': 0.0,
 'sqft_living': 0.0,
 'sqft_lot': 0.0,
 'view': 0.0,
 'waterfront': 0.0,
 'yr_built': 0.0,
 'yr_renovated': 0.0}

***QUIZ QUESTION***

What features had non-zero weight in this case?

Finally, learn the weights with `l1_penalty=1e4`, on the training data. Initialize weights to all zeros, and set the `tolerance=5e5`.  Call resulting weights `weights1e4`, you will need them later.  (This case will take [quite a bit longer](https://xkcd.com/303/) to converge than the others above.)

In [44]:
l1_penalty=1e4
tolerance=5e5
weights1e4 = lasso_cyclical_coordinate_descent(normalized_feature_matrix, output,
                                               initial_weights, l1_penalty, tolerance)

feature_list = ['constant'] + all_features
feature_weights1e4 = dict(zip(feature_list, weights1e4))
feature_weights1e4

{'bathrooms': 14984932.171113443,
 'bedrooms': -19472165.15076051,
 'condition': 3978526.5548528363,
 'constant': 77664061.64562127,
 'floors': -7580888.929524973,
 'grade': 13776609.140049504,
 'sqft_above': -11474194.800730158,
 'sqft_basement': -4874702.397179176,
 'sqft_living': 88451858.26250322,
 'sqft_lot': -2367122.7589984387,
 'view': 7489670.900648955,
 'waterfront': 6722334.518452397,
 'yr_built': -82543644.3015089,
 'yr_renovated': 2943815.787952783}

***QUIZ QUESTION***

What features had non-zero weight in this case?

## Rescaling learned weights

Recall that we normalized our feature matrix, before learning the weights.  To use these weights on a test set, we must normalize the test data in the same way.

Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data: 

In this case, we must scale the resulting weights so that we can make predictions with *original* features:
 1. Store the norms of the original features to a vector called `norms`:
```
features, norms = normalize_features(features)
```
 2. Run Lasso on the normalized features and obtain a `weights` vector
 3. Compute the weights for the original features by performing element-wise division, i.e.
```
weights_normalized = weights / norms
```
Now, we can apply `weights_normalized` to the test data, without normalizing it!

Create a normalized version of each of the weights learned above. (`weights1e4`, `weights1e7`, `weights1e8`).

In [46]:
my_output = 'price'
(feature_matrix, output) = get_numpy_data(train_data, all_features, my_output)
normalized_feature_matrix, norms = normalize_features(feature_matrix)

normalized_weights1e7 = weights1e7 / norms
normalized_weights1e8 = weights1e8 / norms
normalized_weights1e4 = weights1e4 / norms

normalized_weights1e7[3]

161.2951558809334

To check your results, if you call `normalized_weights1e7` the normalized version of `weights1e7`, then:
```
print("{:.4f}".format(normalized_weights1e7[3]))
```
should return 161.2952

## Evaluating each of the learned models on the test data

Let's now evaluate the three models on the test data:

In [47]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, all_features, 'price')

Compute the RSS of each of the three normalized weights on the (unnormalized) `test_feature_matrix`:

In [48]:
prediction =  predict_output(test_feature_matrix, normalized_weights1e7)
RSS = np.dot(test_output-prediction, test_output-prediction)
RSS

271730500343684.94

In [49]:
prediction =  predict_output(test_feature_matrix, normalized_weights1e8)
RSS = np.dot(test_output-prediction, test_output-prediction)
RSS

514904176489231.06

In [50]:
prediction =  predict_output(test_feature_matrix, normalized_weights1e4)
RSS = np.dot(test_output-prediction, test_output-prediction)
RSS

225312368588044.56

***QUIZ QUESTION***

Which model performed best on the test data?
- weights1e4